In [1]:
import cv2
import tensorflow as tf
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, MaxPooling2D, Dropout, BatchNormalization
from keras.utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.utils import compute_class_weight

In [14]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [2]:
dataset_folder = 'dataset_folder'

In [3]:

train_dir = 'dataset_folder/train'
test_dir = 'dataset_folder/test'

train_datagen = ImageDataGenerator(rescale=1./255, rotation_range=10, width_shift_range=0.1, height_shift_range=0.1, shear_range=0.1, zoom_range=0.1, horizontal_flip=True, fill_mode='nearest')
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical'  
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(48, 48),
    color_mode='grayscale',
    batch_size=64,
    class_mode='categorical')

class_weights = compute_class_weight(
                                        class_weight = "balanced",
                                        classes = np.unique(train_generator.classes),
                                        y = train_generator.classes
                                    )
class_weights = dict(zip(np.unique(train_generator.classes), class_weights))
class_weights



Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


{0: 1.0266046844269623,
 1: 9.406618610747051,
 2: 1.0010460615781582,
 3: 0.5684387684387684,
 4: 0.8260394187886635,
 5: 0.8491274770777877,
 6: 1.293372978330405}

In [ ]:
# CNN model
model = Sequential()

# Feature extraction layers
model.add(Conv2D(64, kernel_size=3, input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(128, kernel_size=3))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(256, kernel_size=3))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(512, kernel_size=3))
model.add(BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# MLP
model.add(Flatten())
model.add(Dense(1024, activation='relu'))
model.add(Dense(7, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    validation_data=test_generator, 
    validation_steps=test_generator.samples // test_generator.batch_size,
    epochs=50,
    class_weight=class_weights
)

model.evaluate(test_generator)
model.save('face_model_savedmodel', save_format='tf')

In [ ]:
epochs = range(1, len(history.history['accuracy']) + 1)

plt.figure(figsize=(10, 6))
plt.plot(epochs, history.history['accuracy'], 'bo-', label='Training accuracy')
plt.plot(epochs, history.history['val_accuracy'], 'ro-', label='Testing accuracy')
plt.title('Training and Testing Accuracy over Epochs')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()

plt.show()

In [ ]:
test_steps_per_epoch = np.math.ceil(test_generator.samples / test_generator.batch_size)
predictions = model.predict(test_generator, steps=test_steps_per_epoch)

# Convert predictions classes to one hot vectors
predicted_classes = np.argmax(predictions, axis=1)

# Convert test observations to one hot vectors
true_classes = test_generator.classes

# compute the confusion matrix
conf_matrix = confusion_matrix(true_classes, predicted_classes)

accuracies = conf_matrix.diagonal() / conf_matrix.sum(axis=1)

# compute classification report for each emotion
emotion_labels = list(test_generator.class_indices.keys())
class_report = classification_report(true_classes, predicted_classes, target_names=emotion_labels)

# Display the confusion matrix and classification report
print("Confusion Matrix:")
print(conf_matrix)

print("\nClassification Report:")
print(class_report)

plt.figure(figsize=(6, 4))
plt.bar(emotion_labels, accuracies, color='skyblue')
plt.xlabel('Emotions')
plt.ylabel('Accuracy')
plt.title('Accuracy for Each Emotion')
plt.ylim(0, 0.5)  # Set y-axis to range from 0 to 1
plt.show()

In [ ]:
cap = cv2.VideoCapture(0)

model_directory = 'face_model_savedmodel' 
model = tf.keras.models.load_model(model_directory)

# Define emotion labels
emotion_labels = ['Angry', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

while True:
    
    ret, frame = cap.read()
    if not ret:
        break

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the image
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        
        roi_gray = gray[y:y+h, x:x+w]
        roi_gray = cv2.resize(roi_gray, (48, 48))
        roi = roi_gray.astype('float') / 255.0
        roi = np.expand_dims(roi, axis=0)
        roi = np.expand_dims(roi, axis=-1)

        # Predict the emotion
        preds = model.predict(roi)[0]
        emotion_probability = np.max(preds)
        label = emotion_labels[preds.argmax()]

        label_with_prob = '{}: {:.2f}%'.format(label, emotion_probability * 100)
        
        
        cv2.putText(frame, label_with_prob, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        

    # Display the resulting frame
    cv2.imshow('Video', frame)

    # Exit loop if 'q' is pressed
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()